# 통신사 재무 예측 시스템 - Darts 다중 모델 데모

이 노트북은 **Darts 라이브러리의 다중 모델**을 활용하여 **GL_ACC_LSN_NM(계정과목명) 기준**으로 통신사의 재무 데이터를 분석하여 매출을 예측합니다.

## 🎯 프로젝트 특징
- **계정과목별 예측**: GL_ACC_LSN_NM을 기준으로 한 다변량 시계열 예측
- **5개 모델 지원**: TFT, Prophet, LSTM, GRU, Transformer 모델
- **유연한 앙상블**: 다양한 모델 조합 및 전략 지원
- **동적 특성 엔지니어링**: 시간적 특성, 지연 특성, 이동평균 등

## 📋 목차
1. 환경 설정 및 라이브러리 import
2. 데이터 로드 및 검증
3. 데이터 전처리 및 탐색적 분석
4. 모델 훈련 및 예측
5. 결과 시각화 및 분석
6. 비즈니스 인사이트 도출

## 🤖 지원 모델 상세 가이드

### 1. **TFT (Temporal Fusion Transformer)**
- **특징**: Attention 메커니즘과 시계열 특화 아키텍처 결합
- **장점**:
  - 변수 중요도 해석 가능 (Variable Selection Network)
  - 다중 horizon 예측에 강함  
  - Static, known future, unknown future 변수 모두 처리 가능
  - 어텐션으로 시간 종속성 모델링
- **단점**: 학습 시간이 길고, 하이퍼파라미터 튜닝이 복잡
- **적합한 경우**: 복잡한 패턴과 외부 변수가 많은 비즈니스 데이터

### 2. **Prophet**
- **특징**: Facebook에서 개발한 additive 모델 (trend + seasonality + holidays)
- **장점**:
  - 도메인 지식 반영 용이 (휴일, 이벤트 효과)
  - 누락 데이터 처리 강함
  - 트렌드 변화점 자동 감지
  - 해석 가능한 결과 제공
- **단점**: 복잡한 비선형 패턴 처리 한계
- **적합한 경우**: 명확한 계절성과 트렌드가 있는 단변량 시계열

### 3. **LSTM (Long Short-Term Memory)**
- **특징**: RNN의 vanishing gradient 문제 해결을 위한 게이트 구조
- **장점**:
  - 긴 시퀀스의 장기 종속성 학습 가능
  - 검증된 안정적 성능
  - 다양한 시계열 패턴 처리 가능
- **단점**: 학습 시간이 상대적으로 김
- **적합한 경우**: 긴 의존성이 있는 시계열, 안정적 성능 필요 시

### 4. **GRU (Gated Recurrent Unit)**
- **특징**: LSTM 단순화 버전, reset/update 게이트만 사용
- **장점**:
  - LSTM보다 빠른 훈련 속도
  - 적은 파라미터로 효율적
  - 중간 길이 패턴 학습에 효과적
- **단점**: 매우 긴 시퀀스에서는 LSTM보다 성능 떨어질 수 있음
- **적합한 경우**: 빠른 개발과 적당한 성능이 필요한 경우

### 5. **Transformer**
- **특징**: Self-attention 기반, 병렬 처리 가능한 구조
- **장점**:
  - 병렬 처리로 빠른 훈련
  - Position encoding으로 시간 정보 반영
  - 장거리 종속성 효과적 처리
- **단점**:
  - 많은 데이터 필요
  - 메모리 사용량 높음
- **적합한 경우**: 대용량 데이터, 복잡한 다변량 시계열

## 📊 실무 선택 가이드

| 우선순위 | 모델 | 사용 시나리오 |
|---------|------|-------------|
| 🔍 **해석성 중요** | Prophet | 비즈니스 의사결정 설명 필요 |
| ⚡ **빠른 개발** | GRU | 프로토타이핑, 빠른 결과 필요 |
| 🛡️ **안정적 성능** | LSTM | 검증된 성능, 리스크 최소화 |
| 🎯 **최고 성능 + 해석성** | TFT | 정확도와 설명력 모두 필요 |
| 📈 **대용량 + 복잡 패턴** | Transformer | 충분한 데이터와 컴퓨팅 자원 |

## 📊 데이터 구조
```
BASE_YM, BASE_YY, ENTR_3_PROD_LEVEL_NM, PROFT_SRC_NM, 
GL_ACC_LSN_NO, GL_ACC_LSN_NM, SUM_DIV_NM, PRFIT_PERSP_1_INDX_VAL
```

## 1. 환경 설정 및 라이브러리 import

이 섹션에서는 분석에 필요한 라이브러리들을 import하고 기본 설정을 진행합니다.

### 📦 주요 라이브러리:
- **pandas, numpy**: 데이터 처리 및 수치 연산
- **matplotlib, seaborn**: 데이터 시각화
- **Darts 모델**: TFT, Prophet, LSTM, GRU, Transformer
- **프로젝트 모듈**: data_processor, models, visualizer

### ⚙️ 설정 확인:
- 한글 폰트 설정
- 그래프 크기 기본값 설정  
- config.yaml 파일에서 모델 전략 확인

In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import yaml
from tqdm import tqdm
import time
warnings.filterwarnings('ignore')

# 프로젝트 모듈 import
import sys
sys.path.append('../src')

from data_processor import TelecomDataProcessor
from models import TelecomForecaster
from visualizer import TelecomVisualizer

# 한글 폰트 설정 (한글 출력 시)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ 라이브러리 import 완료")
print("📦 지원 모델: TFT, Prophet, LSTM, GRU, Transformer")

# 설정 파일 확인
try:
    with open('../config/config.yaml', 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    strategy = config['model']['strategy']
    print(f"🎯 현재 모델 전략: {strategy}")
except Exception as e:
    print(f"⚠️ 설정 파일을 확인하세요: config/config.yaml")
    print(f"오류: {e}")

## 2. 데이터 로드 및 검증

이 섹션에서는 통신사 재무 데이터를 로드하고 기본적인 검증을 수행합니다.

### 📊 데이터 요구사항:
- **파일 형식**: CSV, Excel (.xlsx, .xls), DRM 보호 파일 지원
- **필수 컬럼**: BASE_YM, GL_ACC_LSN_NM, PRFIT_PERSP_1_INDX_VAL 등
- **데이터 품질**: 결측치 확인, 시계열 연속성 검증

### 🔍 처리 과정:
1. 데이터 처리기 (TelecomDataProcessor) 생성
2. 파일 존재 여부 확인
3. 전체 데이터 처리 파이프라인 실행
4. 특성 정보 및 기본 통계 확인

In [ ]:
# 데이터 처리기 생성
processor = TelecomDataProcessor(config_path="../config/config.yaml")

# 데이터 파일 경로 설정 
# 지원 파일 형식: CSV, Excel (.xlsx, .xls), DRM 보호 파일 포함
file_path = '../data/raw/telecom_financial_data.xlsx'  # 실제 파일명으로 변경하세요

print(f"📁 데이터 파일 경로: {file_path}")
print("📋 필수 컬럼: BASE_YM, BASE_YY, ENTR_3_PROD_LEVEL_NM, PROFT_SRC_NM")
print("          GL_ACC_LSN_NO, GL_ACC_LSN_NM, SUM_DIV_NM, PRFIT_PERSP_1_INDX_VAL")

# 파일 존재 확인
import os
if os.path.exists(file_path):
    print("✅ 파일 존재 확인")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
    print("📝 data/raw/ 폴더에 통신사 재무 데이터 파일을 넣고 경로를 수정하세요.")
    print("💡 지원 형식: .csv, .xlsx, .xls (DRM 보호 파일 포함)")

In [ ]:
# 데이터 로드 및 처리
try:
    print("🔄 데이터 처리 시작...")
    
    # 전체 데이터 처리 파이프라인 실행
    processed_data = processor.process_data(file_path='../data/raw/telecom_financial_data.xlsx')
    
    print("✅ 데이터 처리 완료")
    print(f"📊 처리된 데이터 크기: {processed_data.shape}")
    print(f"📅 기간: {processed_data.index.min()} ~ {processed_data.index.max()}")
    
    # 특성 정보 확인
    feature_info = processor.get_feature_info()
    print(f"📈 처리된 계정과목: {len(feature_info['account_columns'])}개")
    print(f"🏷️ 처리된 제품: {len(feature_info['product_columns'])}개")
    
except Exception as e:
    print(f"❌ 데이터 처리 실패: {e}")
    print("📝 파일 경로와 형식을 확인하세요.")
    print("💡 USER_GUIDE.md 파일을 참고하여 데이터 형식을 확인하세요.")
    
    # 디버깅 정보
    import traceback
    print(f"\n🔍 상세 오류 정보:")
    print(traceback.format_exc())

## 3. 데이터 전처리 및 탐색적 분석

이 섹션에서는 처리된 데이터를 탐색하고 시각화를 통해 패턴을 분석합니다.

### 📈 분석 내용:
- **데이터 검증**: 처리된 데이터의 구조와 품질 확인
- **계정과목별 매출 추이**: 상위 계정과목들의 시계열 패턴 분석  
- **상관관계 분석**: 계정과목 간의 상관관계 매트릭스
- **기초 통계**: 데이터 분포 및 특성 파악

### 🎯 목적:
- 데이터의 계절성, 트렌드 패턴 파악
- 예측 모델 선택을 위한 데이터 특성 이해
- 비즈니스 인사이트 도출을 위한 기초 분석

In [ ]:
# 데이터 탐색 및 검증
if 'processed_data' in locals():
    print("📊 데이터 검증 및 탐색")
    print("=" * 50)
    
    # 데이터 미리보기
    print("\n📋 처리된 데이터 미리보기:")
    display(processed_data.head())
    
    print(f"\n📈 데이터 정보:")
    print(f"   - 총 행 수: {len(processed_data):,}")
    print(f"   - 총 컬럼 수: {len(processed_data.columns):,}")
    print(f"   - 결측치: {processed_data.isnull().sum().sum():,}개")
    print(f"   - 메모리 사용량: {processed_data.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # 주요 계정과목 확인
    if 'feature_info' in locals():
        print(f"\n🎯 주요 계정과목 (상위 10개):")
        for i, col in enumerate(feature_info['account_columns'][:10], 1):
            print(f"   {i:2d}. {col}")
    
    # 기초 통계 정보
    print(f"\n📊 수치형 데이터 기초 통계:")
    display(processed_data.describe())
else:
    print("❌ 처리된 데이터가 없습니다. 이전 셀을 먼저 실행하세요.")

In [ ]:
# 계정과목별 매출 추이 시각화
if 'processed_data' in locals() and 'feature_info' in locals():
    print("📊 계정과목별 매출 추이 분석")
    
    # 상위 6개 계정과목 시각화
    top_accounts = feature_info['account_columns'][:6]
    
    plt.figure(figsize=(15, 10))
    
    for i, account in enumerate(top_accounts, 1):
        if account in processed_data.columns:
            plt.subplot(2, 3, i)
            
            # 매출 추이 플롯
            processed_data[account].plot(linewidth=2, color=f'C{i-1}')
            plt.title(f'{account}', fontsize=12, fontweight='bold')
            plt.xlabel('날짜')
            plt.ylabel('매출액')
            plt.grid(True, alpha=0.3)
            
            # 최신값 표시
            latest_value = processed_data[account].iloc[-1]
            plt.text(0.02, 0.98, f'최신: {latest_value:,.0f}', 
                    transform=plt.gca().transAxes, 
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # 매출 합계 추이
    plt.figure(figsize=(12, 6))
    total_revenue = processed_data[feature_info['account_columns']].sum(axis=1)
    total_revenue.plot(linewidth=2, color='darkblue')
    plt.title('전체 계정과목 매출 합계 추이', fontsize=14, fontweight='bold')
    plt.xlabel('날짜')
    plt.ylabel('총 매출액')
    plt.grid(True, alpha=0.3)
    plt.show()
    
else:
    print("❌ 데이터가 준비되지 않았습니다. 이전 셀을 먼저 실행하세요.")

In [ ]:
# 계정과목별 상관관계 분석
if 'processed_data' in locals() and 'feature_info' in locals():
    print("🔗 계정과목별 상관관계 분석")
    
    # 상위 10개 계정과목의 상관관계 매트릭스
    top_10_accounts = feature_info['account_columns'][:10]
    correlation_data = processed_data[top_10_accounts]
    
    plt.figure(figsize=(12, 10))
    correlation_matrix = correlation_data.corr()
    
    # 히트맵 생성
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, 
                mask=mask,
                annot=True, 
                cmap='RdYlBu_r', 
                center=0,
                square=True,
                fmt='.2f',
                cbar_kws={"shrink": .8})
    
    plt.title('계정과목별 상관관계 매트릭스 (상위 10개)', fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # 높은 상관관계를 가진 계정과목 쌍 찾기
    print("\n🔍 높은 상관관계를 가진 계정과목 쌍 (상관계수 > 0.7):")
    high_corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_value = correlation_matrix.iloc[i, j]
            if abs(corr_value) > 0.7:
                account1 = correlation_matrix.columns[i]
                account2 = correlation_matrix.columns[j]
                high_corr_pairs.append((account1, account2, corr_value))
                print(f"   📊 {account1} ↔ {account2}: {corr_value:.3f}")
    
    if not high_corr_pairs:
        print("   ℹ️ 상관계수 0.7 이상인 계정과목 쌍이 없습니다.")
else:
    print("❌ 데이터가 준비되지 않았습니다. 이전 셀을 먼저 실행하세요.")

## 4. 모델 훈련 및 예측

이 섹션에서는 Darts 라이브러리의 다중 모델을 사용하여 예측을 수행합니다.

### 🤖 모델 설정:
- **예측기 생성**: TelecomForecaster 인스턴스 생성
- **타겟 선택**: 상위 10개 계정과목 선택
- **예측 기간**: 12개월 미래 예측
- **모델 전략**: config.yaml에서 설정된 전략 사용

### 🎯 지원 전략:
- **tft_only**: TFT 모델만 사용 (빠른 실행)
- **ensemble**: TFT + Prophet 조합 (안정적)
- **multi_model**: 5개 모델 앙상블 (최고 성능)
- **auto_select**: 성능 기반 자동 선택

### ⚡ 실행 참고:
- GPU 사용 시 훈련 속도 향상
- 모델 전략에 따라 5분~60분 소요
- 메모리 부족 시 config.yaml에서 배치 크기 조정

In [ ]:
# 예측기 생성 및 설정
if 'processed_data' in locals() and 'feature_info' in locals():
    print("🤖 예측 모델 설정")
    print("=" * 50)
    
    # 예측기 생성
    forecaster = TelecomForecaster(config_path="../config/config.yaml")
    
    # 예측 대상 계정과목 선택 (상위 10개)
    target_columns = feature_info['account_columns'][:10]
    forecast_horizon = 12  # 12개월 예측
    
    print(f"🎯 예측 대상 계정과목 ({len(target_columns)}개):")
    for i, col in enumerate(target_columns, 1):
        print(f"   {i:2d}. {col}")
    
    print(f"\n📅 예측 기간: {forecast_horizon}개월")
    
    # 현재 모델 전략 확인
    if 'config' in locals():
        strategy = config['model']['strategy']
        print(f"🧠 모델 전략: {strategy}")
        
        if strategy == "tft_only":
            print("   📊 TFT 모델만 사용 (빠른 실행)")
        elif strategy == "ensemble":
            models = config['model']['ensemble']['models']
            weights = config['model']['ensemble']['weights']
            print(f"   🔗 앙상블 모델: {models}")
            print(f"   ⚖️ 가중치: {weights}")
        elif strategy == "multi_model":
            models = config['model']['multi_model_ensemble']['models']
            print(f"   🎭 다중 모델: {models}")
            print("   ⚠️ 실행 시간이 오래 걸릴 수 있습니다 (30-60분)")
    
    # GPU 사용 가능 여부 확인
    try:
        import torch
        if torch.cuda.is_available():
            print("\n🚀 GPU 사용 가능 - 빠른 훈련 예상")
            print(f"   GPU 모델: {torch.cuda.get_device_name()}")
        else:
            print("\n💻 CPU 사용 - 훈련 시간이 오래 걸릴 수 있습니다")
    except ImportError:
        print("\n💻 PyTorch 미설치 - CPU 사용")
        
else:
    print("❌ 데이터가 준비되지 않았습니다. 이전 셀들을 먼저 실행하세요.")

In [ ]:
# 예측 파이프라인 실행
if 'forecaster' in locals() and 'target_columns' in locals():
    try:
        print("🔄 예측 파이프라인 실행 중...")
        print("⏰ 모델 전략에 따라 시간이 오래 걸릴 수 있습니다...")
        print("💡 실행 중 다른 작업을 하셔도 됩니다.")
        
        # 진행바 추가 (간단 버전)
        with tqdm(total=100, desc="예측 진행률", 
                 bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]") as pbar:
            
            # 예측 파이프라인 실행 (progress_callback 제거)
            results = forecaster.run_forecast_pipeline(
                processed_data=processed_data,
                target_columns=target_columns,
                forecast_horizon=forecast_horizon
            )
            
            # 완료 시 진행바 100%로 설정
            pbar.update(100 - pbar.n)
        
        print("\n✅ 예측 완료!")
        print("�� 결과 파일이 results/ 폴더에 저장되었습니다.")
        
        # 결과 요약
        if 'ensemble_forecast' in results:
            print(f"\n📊 예측 결과 요약:")
            print(f"   - 예측 대상: {len(target_columns)}개 계정과목")
            print(f"   - 예측 기간: {forecast_horizon}개월")
            print(f"   - 예측 시작일: {results['ensemble_forecast'].index[0]}")
            print(f"   - 예측 종료일: {results['ensemble_forecast'].index[-1]}")
        
        if 'evaluation_results' in results:
            print(f"\n🎯 모델 성능 (MAPE 기준):")
            eval_results = results['evaluation_results']
            for model_name in eval_results.keys():
                avg_mape = np.mean([metrics.get('mape', 0) for metrics in eval_results[model_name].values()])
                print(f"   {model_name.upper()}: {avg_mape:.2f}%")
        
    except Exception as e:
        print(f"\n❌ 예측 실패: {e}")
        print("📝 다음을 확인해주세요:")
        print("   1. 데이터 파일이 올바른 형식인지")
        print("   2. 메모리가 충분한지")
        print("   3. 설정 파일 (config/config.yaml)이 올바른지")
        
else:
    print("❌ 예측기가 준비되지 않았습니다. 이전 셀들을 먼저 실행하세요.")

## 5. 결과 시각화 및 분석

이 섹션에서는 예측 결과를 시각화하고 상세 분석을 수행합니다.

### 📊 시각화 내용:
- **예측 결과 확인**: 앙상블 예측값과 개별 모델 결과
- **과거 vs 예측 비교**: 실제 데이터와 예측 데이터 시각화
- **모델 성능 평가**: MAPE, MAE 등 정확도 지표 비교
- **계정과목별 성장 분석**: 성장률 및 트렌드 분석

### 🎯 분석 관점:
- **정확도**: 각 모델의 예측 성능 비교
- **안정성**: 예측 결과의 일관성 평가  
- **해석성**: 비즈니스 관점에서의 결과 해석
- **실용성**: 의사결정에 활용 가능한 인사이트 도출

### 📈 출력 결과:
- 상위 6개 계정과목별 예측 차트
- 전체 매출 합계 예측 트렌드
- 성장률 분포 및 우선순위 분석

In [ ]:
# 예측 결과 확인 및 분석
if 'results' in locals():
    print("📊 예측 결과 상세 분석")
    print("=" * 50)
    
    # 앙상블 예측 결과 확인
    if 'ensemble_forecast' in results:
        forecast_df = results['ensemble_forecast']
        print(f"\n🔮 앙상블 예측 결과:")
        print(f"   - 예측 데이터 크기: {forecast_df.shape}")
        print(f"   - 예측 기간: {forecast_df.index[0]} ~ {forecast_df.index[-1]}")
        
        print(f"\n📋 예측 결과 미리보기 (처음 5개월):")
        display(forecast_df.head())
        
        print(f"\n📈 12개월 후 예측값:")
        final_predictions = forecast_df.iloc[-1]
        for col in target_columns[:5]:  # 상위 5개만 표시
            if col in final_predictions.index:
                value = final_predictions[col]
                print(f"   📊 {col}: {value:,.0f}원")
    
    # 개별 모델 예측 결과
    if 'individual_forecasts' in results:
        print(f"\n🎭 개별 모델 예측 결과:")
        individual_forecasts = results['individual_forecasts']
        for model_name, forecast in individual_forecasts.items():
            print(f"   {model_name.upper()}: {forecast.shape[0]}개월 예측")
    
    # 모델 성능 평가
    if 'evaluation_results' in results:
        print(f"\n🎯 모델별 성능 평가 (주요 계정과목):")
        eval_results = results['evaluation_results']
        
        for account in target_columns[:3]:  # 상위 3개만 표시
            print(f"\n   📊 {account}:")
            for model_name, model_metrics in eval_results.items():
                if account in model_metrics:
                    metrics = model_metrics[account]
                    mape = metrics.get('mape', 'N/A')
                    mae = metrics.get('mae', 'N/A')
                    if isinstance(mape, (int, float)):
                        print(f"      {model_name.upper()}: MAPE={mape:.2f}%, MAE={mae:,.0f}")
                    else:
                        print(f"      {model_name.upper()}: MAPE={mape}, MAE={mae}")
else:
    print("❌ 예측 결과가 없습니다. 이전 셀을 먼저 실행하세요.")

In [ ]:
# 예측 결과 시각화
if 'results' in locals() and 'ensemble_forecast' in results:
    print("📊 예측 결과 시각화")
    
    # 상위 6개 계정과목에 대한 예측 결과 시각화
    forecast_df = results['ensemble_forecast']
    top_6_accounts = target_columns[:6]
    
    plt.figure(figsize=(18, 12))
    
    for i, account in enumerate(top_6_accounts, 1):
        if account in forecast_df.columns and account in processed_data.columns:
            plt.subplot(2, 3, i)
            
            # 과거 데이터 (최근 24개월)
            historical_data = processed_data[account].iloc[-24:]
            plt.plot(historical_data.index, historical_data.values, 
                    label='과거 데이터', linewidth=2, color='blue')
            
            # 예측 데이터
            plt.plot(forecast_df.index, forecast_df[account].values, 
                    label='예측 데이터', linewidth=2, linestyle='--', color='red')
            
            # 예측 시작점 표시
            plt.axvline(x=historical_data.index[-1], color='gray', linestyle=':', alpha=0.7)
            
            plt.title(f'{account}', fontsize=12, fontweight='bold')
            plt.xlabel('날짜')
            plt.ylabel('매출액')
            plt.legend(fontsize=8)
            plt.grid(True, alpha=0.3)
            
            # 최근 실제값과 12개월 후 예측값 표시
            recent_actual = historical_data.iloc[-1]
            future_prediction = forecast_df[account].iloc[-1]
            growth_rate = ((future_prediction - recent_actual) / recent_actual) * 100
            
            plt.text(0.02, 0.98, f'현재: {recent_actual:,.0f}\n12개월 후: {future_prediction:,.0f}\n성장률: {growth_rate:+.1f}%', 
                    transform=plt.gca().transAxes, 
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                    fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # 전체 매출 합계 예측
    plt.figure(figsize=(14, 6))
    
    # 과거 전체 매출
    historical_total = processed_data[target_columns].sum(axis=1).iloc[-24:]
    plt.plot(historical_total.index, historical_total.values, 
            label='과거 전체 매출', linewidth=3, color='darkblue')
    
    # 예측 전체 매출
    forecast_total = forecast_df[target_columns].sum(axis=1)
    plt.plot(forecast_total.index, forecast_total.values, 
            label='예측 전체 매출', linewidth=3, linestyle='--', color='darkred')
    
    plt.axvline(x=historical_total.index[-1], color='gray', linestyle=':', alpha=0.7, label='예측 시작점')
    
    plt.title('전체 계정과목 매출 합계 예측', fontsize=16, fontweight='bold')
    plt.xlabel('날짜')
    plt.ylabel('총 매출액')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 총 매출 성장률 표시
    current_total = historical_total.iloc[-1]
    future_total = forecast_total.iloc[-1]
    total_growth = ((future_total - current_total) / current_total) * 100
    
    plt.text(0.02, 0.98, f'현재 총 매출: {current_total:,.0f}원\n12개월 후 예상: {future_total:,.0f}원\n전체 성장률: {total_growth:+.1f}%', 
            transform=plt.gca().transAxes, 
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9),
            fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ 예측 결과가 없습니다. 이전 셀을 먼저 실행하세요.")

## 6. 비즈니스 인사이트 도출

이 섹션에서는 예측 결과를 바탕으로 비즈니스 관점의 인사이트와 전략적 권장사항을 제시합니다.

### 💡 분석 내용:
- **계정과목별 성장 분석**: 고성장 vs 주의필요 계정과목 식별
- **성장률 분포**: 12개월 후 예상 성장률 시각화
- **매출 규모별 전략**: 대형/중형/소형 계정과목별 접근법
- **시각화 리포트 생성**: HTML 대시보드 및 차트 파일 생성

### 🎯 전략적 권장사항:
- **고성장 계정과목**: 마케팅 투자 확대, 신규 서비스 개발
- **성장 둔화 계정과목**: 서비스 혁신, 비용 구조 최적화
- **수익성 개선**: ARPU 향상, 고객 세분화, 운영 효율성 개선
- **모니터링 체계**: 월별 실적 추적, 분기별 모델 재훈련

### 📊 활용 방안:
- results/ 폴더의 HTML 리포트 확인
- dashboard.html로 종합 현황 파악  
- 정기적인 예측 업데이트 및 전략 수정

In [ ]:
# 계정과목별 성장 분석 및 인사이트
if 'results' in locals() and 'ensemble_forecast' in results:
    print("💡 계정과목별 성장 분석 및 비즈니스 인사이트")
    print("=" * 60)
    
    forecast_df = results['ensemble_forecast']
    
    # 계정과목별 성장률 분석
    growth_analysis = []
    
    for account in target_columns:
        if account in forecast_df.columns and account in processed_data.columns:
            # 현재값 (최신 실제 데이터)
            current_value = processed_data[account].iloc[-1]
            
            # 12개월 후 예측값
            future_value = forecast_df[account].iloc[-1]
            
            # 성장률 계산
            growth_rate = ((future_value - current_value) / current_value) * 100
            
            # 연간 매출 규모 (현재 기준)
            annual_revenue = current_value * 12
            
            growth_analysis.append({
                'account': account,
                'current_value': current_value,
                'future_value': future_value,
                'growth_rate': growth_rate,
                'annual_revenue': annual_revenue
            })
    
    # 데이터프레임으로 변환 및 정렬
    growth_df = pd.DataFrame(growth_analysis)
    growth_df = growth_df.sort_values('growth_rate', ascending=False)
    
    print(f"\n🚀 고성장 계정과목 (성장률 상위 5개):")
    top_growth = growth_df.head(5)
    for idx, row in top_growth.iterrows():
        print(f"   📈 {row['account']}")
        print(f"      현재 매출: {row['current_value']:,.0f}원")
        print(f"      12개월 후: {row['future_value']:,.0f}원")
        print(f"      성장률: {row['growth_rate']:+.1f}%")
        print(f"      연간 규모: {row['annual_revenue']:,.0f}원")
        print()
    
    print(f"⚠️ 주의 필요 계정과목 (성장률 하위 5개):")
    bottom_growth = growth_df.tail(5)
    for idx, row in bottom_growth.iterrows():
        print(f"   📉 {row['account']}")
        print(f"      현재 매출: {row['current_value']:,.0f}원")
        print(f"      12개월 후: {row['future_value']:,.0f}원")
        print(f"      성장률: {row['growth_rate']:+.1f}%")
        if row['growth_rate'] < -5:
            print(f"      🔴 전략 재검토 필요")
        elif row['growth_rate'] < 0:
            print(f"      🟡 주의 깊은 모니터링 필요")
        print()
    
    # 성장률 분포 시각화
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    colors = ['green' if x > 5 else 'orange' if x > 0 else 'red' for x in growth_df['growth_rate']]
    bars = plt.bar(range(len(growth_df)), growth_df['growth_rate'], color=colors, alpha=0.7)
    plt.title('계정과목별 12개월 성장률 예측', fontsize=14, fontweight='bold')
    plt.ylabel('성장률 (%)')
    plt.xticks(range(len(growth_df)), growth_df['account'], rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    # 성장률별 색상 범례
    plt.text(0.02, 0.98, '🟢 5% 이상: 고성장\n🟡 0-5%: 안정성장\n🔴 음수: 감소', 
            transform=plt.gca().transAxes, 
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
    
    plt.subplot(2, 1, 2)
    # 매출 규모별 성장률 산점도
    plt.scatter(growth_df['annual_revenue'], growth_df['growth_rate'], 
               c=colors, alpha=0.7, s=100)
    plt.xlabel('연간 매출 규모 (원)')
    plt.ylabel('성장률 (%)')
    plt.title('매출 규모 vs 성장률', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    # 주요 계정과목 레이블 표시
    for idx, row in growth_df.head(3).iterrows():
        plt.annotate(row['account'], 
                    (row['annual_revenue'], row['growth_rate']),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=8, alpha=0.8)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ 예측 결과가 없습니다. 이전 셀을 먼저 실행하세요.")

In [ ]:
# 시각화 리포트 생성 및 전략적 권장사항
if 'results' in locals():
    print("📊 시각화 리포트 생성")
    print("=" * 50)
    
    try:
        # 시각화기 생성
        visualizer = TelecomVisualizer(config_path="../config/config.yaml")
        
        # 종합 리포트 생성
        report_files = visualizer.generate_report(
            processed_data=processed_data,
            results=results,
            target_columns=target_columns
        )
        
        print("✅ 시각화 리포트 생성 완료!")
        print("\n📁 생성된 파일들:")
        for file_path in report_files:
            print(f"   📄 {file_path}")
        
        print("\n💡 리포트 활용 방법:")
        print("   1. 📊 forecast_plot.html: 예측 결과 차트")
        print("   2. 📈 accuracy_plot.html: 모델 정확도 비교") 
        print("   3. 🔗 seasonal_plot.html: 계절성 분석")
        print("   4. 📋 dashboard.html: 종합 대시보드")
        
    except Exception as e:
        print(f"⚠️ 리포트 생성 중 오류 발생: {e}")
        print("📝 수동으로 결과를 확인하겠습니다.")
    
    print(f"\n🎯 전략적 권장사항")
    print("=" * 50)
    
    if 'ensemble_forecast' in results:
        forecast_df = results['ensemble_forecast']
        
        # 전체 매출 성장률 계산
        current_total = processed_data[target_columns].sum(axis=1).iloc[-1]
        future_total = forecast_df[target_columns].sum(axis=1).iloc[-1] 
        total_growth = ((future_total - current_total) / current_total) * 100
        
        print(f"\n📈 전체 매출 성장 전망:")
        print(f"   현재 월 매출: {current_total:,.0f}원")
        print(f"   12개월 후 예상: {future_total:,.0f}원")  
        print(f"   전체 성장률: {total_growth:+.1f}%")
        
        if total_growth > 5:
            print(f"   💚 강한 성장 예상 - 적극적 투자 권장")
        elif total_growth > 0:
            print(f"   📈 안정적 성장 예상 - 현 전략 유지")
        else:
            print(f"   ⚠️ 성장 둔화 예상 - 전략 재검토 필요")
    
    print(f"\n🏢 사업 영역별 전략:")
    print(f"   📊 고성장 계정과목:")
    print(f"      ✅ 마케팅 투자 확대")
    print(f"      ✅ 신규 서비스 개발")
    print(f"      ✅ 고객 획득 프로그램 강화")
    
    print(f"\n   📉 성장 둔화 계정과목:")
    print(f"      🔄 서비스 혁신 필요")
    print(f"      🔄 비용 구조 최적화")  
    print(f"      🔄 대체 수익원 발굴")
    
    print(f"\n💰 수익성 개선 방안:")
    print(f"   📈 ARPU 향상을 위한 부가서비스 개발")
    print(f"   📈 고객 세분화를 통한 맞춤형 서비스")
    print(f"   📈 운영 효율성 개선으로 비용 절감")
    print(f"   📈 신규 수익 모델 및 파트너십 발굴")
    
    print(f"\n🎯 모니터링 권장사항:")
    print(f"   📅 월별 실적 vs 예측 비교 분석")
    print(f"   📊 주요 KPI 대시보드 운영")
    print(f"   🔄 분기별 모델 재훈련 및 예측 업데이트")
    print(f"   📈 외부 환경 변화에 따른 예측 조정")
    
else:
    print("❌ 예측 결과가 없습니다. 이전 셀을 먼저 실행하세요.")

## 📝 결론

이 노트북을 통해 **Darts 라이브러리의 다중 모델**을 활용하여 **GL_ACC_LSN_NM(계정과목명) 기준**으로 통신사의 재무 데이터를 분석하고 매출을 예측했습니다.

## 🎯 주요 성과

### 📊 예측 시스템 특징:
- **5개 모델 지원**: TFT, Prophet, LSTM, GRU, Transformer 
- **유연한 앙상블**: 모델별 가중치 조합으로 정확도 향상
- **계정과목별 예측**: GL_ACC_LSN_NM 기준 세밀한 매출 예측
- **동적 특성 엔지니어링**: 시간적 특성, 지연 특성, 이동평균

### 🔍 분석 결과:
- 📈 계정과목별 12개월 매출 예측 완료
- 🎯 고성장/주의필요 계정과목 식별
- 📊 모델별 성능 비교 및 최적 조합 도출
- 💡 데이터 기반 비즈니스 인사이트 제공

## 📁 생성된 결과물

### 데이터 파일:
- `results/forecast_results.csv`: 상세 예측 결과
- `results/evaluation_results.csv`: 모델 성능 평가

### 시각화 리포트:
- `results/forecast_plot.html`: 📊 예측 결과 차트
- `results/accuracy_plot.html`: 📈 모델 정확도 비교  
- `results/seasonal_plot.html`: 🔄 계절성 분석
- `results/dashboard.html`: 📋 종합 대시보드

## 🎯 다음 단계

### 즉시 실행:
1. 📁 `results/` 폴더에서 HTML 리포트 확인
2. 📊 `dashboard.html`로 종합 현황 파악
3. 📈 주요 계정과목 성장률 분석 결과 검토

### 정기 운영:
1. 🔄 **월별 실적 업데이트**: 새 데이터로 예측 정확도 검증
2. 📅 **분기별 모델 재훈련**: 최신 트렌드 반영으로 정확도 유지
3. 🎯 **전략 수정**: 예측 결과 기반 비즈니스 전략 조정
4. 📊 **KPI 모니터링**: 실제 실적 vs 예측 비교 분석

## 🛠️ 추가 활용

### 고급 분석:
- **시나리오 분석**: 다양한 비즈니스 상황별 예측
- **민감도 분석**: 주요 변수 변화에 따른 영향 분석  
- **리스크 분석**: 예측 불확실성 및 리스크 요소 평가

### 모델 개선:
- **하이퍼파라미터 튜닝**: 모델 성능 최적화
- **외부 변수 추가**: 경제지표, 경쟁사 데이터 등 반영
- **앙상블 전략 개선**: 모델 조합 및 가중치 최적화

---

📚 **참고 문서**: 
- [USER_GUIDE.md](../USER_GUIDE.md): 상세한 사용법
- [MODEL_USAGE_GUIDE.md](../MODEL_USAGE_GUIDE.md): 모델 선택 가이드
- [README.md](../README.md): 프로젝트 전체 개요

🎉 **분석 완료!** Darts 라이브러리의 다중 모델 예측 결과를 활용하여 데이터 기반 의사결정을 진행하세요.